In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from skimage.feature import hog
import matplotlib.pyplot as plt

In [ ]:
cohis = 8
grhis = 24

In [ ]:
def read(name):
    image = cv2.imread(name)
    b,g,r = cv2.split(image)
    gray = cv2.cvtColor(cv2.resize(image, (100,100)), cv2.COLOR_RGB2GRAY)
    
    histogram = np.concatenate([np.histogram(i, bins=cohis, range=(0,256))[0] for i in [b,g,r]])
    full = hog(gray, orientations=grhis, pixels_per_cell=(10,10), cells_per_block=(1,1))
    gradient = full.reshape(100, grhis).sum(0)
    return np.concatenate([histogram, gradient])

def load(folder):
    directory = os.listdir(folder)
    if '.DS_Store' in directory:
        directory.remove('.DS_Store')
    df = pd.DataFrame(data=[read(f'{folder}/{each}') for each in directory])
    df.loc[:, 'type'] = folder
    return df

In [ ]:
from sklearn.utils import shuffle
nature = load('nature')
city = load('city')
data = shuffle(pd.concat([i.sample(550) for i in [city, nature]]))

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
y = enc.fit_transform(data.loc[:, 'type'])

In [ ]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
x = scl.fit_transform(data.iloc[:, :-1])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.1)

In [ ]:
from sklearn.svm import SVC
lab = SVC(C=1.5)
lab.fit(x_train, y_train)
lab.score(x_test, y_test)

In [ ]:
test = read('city.jpg') # change the image name
result = lab.predict(scl.transform([test]))[0]
enc.inverse_transform([result])[0]